In [1]:
import osmnx as ox
import networkx as nx
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import random
import simpy
import pandas as pd
import os
import pickle
import time
from shapely.geometry import LineString, Point

In [33]:
# First define some global variables
class GV:
    SIM_TIME = 100 + 1
    GEN_RATE = 1.5
    #GEN_END = (SIM_TIME - 1) * GEN_RATE * 0.9
    ATK_RATE = None

    MOVE_INTV = 0.2
    VIS_INTV = None
    REROUTE_INTV = None

    VEHICLE_LENGTH = 4.5
    WARMING_UP = 0

    BETA_1 = 1
    BETA_2 = 3
    BETA_3 = 20
    CP_1 = 0.5
    CP_2 = 0.8
    NEXT_CP = 0.5
    
    SEED = 42
    IMG_DIR = os.getcwd() + '\\seed-'+ str(SEED) + '_dur-' + str(SIM_TIME) + '_gen-' + str(GEN_RATE) + \
                '_rr-' + str(REROUTE_INTV) + '_atk-' + str(ATK_RATE) + '_warm_up-' + str(WARMING_UP) +  '\\'
    os.makedirs(IMG_DIR)
    
    links = [
        'Motorway_link', 'Primary_link',
        'Secondary_link', 'Tertiary_link'
    ]

# Traffic generation process
class Traffic_Gen(object): 
    def __init__(self, env, G):
        self.env = env
        self.gen_rate = GV.GEN_RATE
        self.vehicle_number = 0
        self.G = G
        self.nodes = list(self.G.nodes)
        self.weights = [self.G.nodes[node]['aggr_node_num'] for node in self.nodes]

        self.Q_dic = {}   # Dictionary of traffic queue for each edge in the network 
        self.delay_dic = {}   # Dictionary of practical delay log on each edge
        
        for edge in self.G.edges:
            self.Q_dic[edge] = []
            self.delay_dic[edge] = []

        self.action = env.process(self.run())
        
    def run(self):       
        while True:
            # Infinite loop for generating traffic
            yield self.env.timeout(random.expovariate(self.gen_rate))
                           
            #if self.vehicle_number >= GV.GEN_END : continue
            
            # Create and enqueue a new vehicle
            gen_time = self.env.now  
            
            generated = False
            gen_trial = 0
            
            while not generated:
                try :
                    # Generate a vehicle with random src/dst pair
                    src_dst = random.choices(self.nodes, weights=self.weights, k=2)
                    src = src_dst[0]
                    dst = src_dst[1]
                    
                    if src == dst:
                        raise ValueError('src and dst node are the same')
                        
                    path = nx.shortest_path(self.G, src, dst, weight='expected_delay')
                    start_edge = (path[0], path[1], 0)

                    new_vehicle = Vehicle(self.vehicle_number, self.env.now, src, dst, path)

                    # Put the vehicle in the starting edge
                    self.vehicle_entry(start_edge, new_vehicle)
                    self.G.edges[start_edge]['edge_cnt'] += 1
                    
                    self.vehicle_number += 1
                    generated = True

                except Exception as error:
                    # Errors above and there are some pairs that have no path between src/dst
                    #print(error)
                    pass
            
    def vehicle_entry(self, edge, vehicle):
        # Add vehicle to the queue of selected edge
        q = self.Q_dic[edge]
        q.append(vehicle)
        vehicle.entry_time = self.env.now
        vehicle.edge_delay = self.G.edges[edge]['total_delay']
        vehicle.edge_sat = self.G.edges[edge]['saturation']
        
        # Update peak traffic and saturation rate of the edge
        trf_len = len(q)
        if trf_len > self.G.edges[edge]['peak_traffic']:
            self.G.edges[edge]['peak_traffic'] = trf_len
        self.G.edges[edge]['saturation'] = (trf_len * GV.VEHICLE_LENGTH) / (self.G.edges[edge]['length'] * self.G.edges[edge]['lanes'])
        
        # Update edge delay
        self.update_delay(edge)
    
    def update_delay(self, edge):
        base_delay = self.G.edges[edge]['travel_time']
        signal_delay = 0
        congest_delay = 0

        edge_type = self.G.edges[edge]['highway']
        saturation = self.G.edges[edge]['saturation']
        edge_len = self.G.edges[edge]['length']
        beta_1 = GV.BETA_1
        beta_2 = GV.BETA_2
        beta_3 = GV.BETA_3
        cp1 = GV.CP_1
        cp2 = GV.CP_2

        if edge_type == 'primary':
            signal_delay = 10

        elif edge_type == 'secondary':
            signal_delay = 10

        elif edge_type == 'tertiary':
            signal_delay = 6
                
        elif edge_type == 'residential':
            signal_delay = 4

        # Get penalty rate    

        # Is any of possible next edges congested?
        next_edge_congested = False
        (current_node, next_node, key) = edge

        for (u, v) in self.G.out_edges(next_node):
            if v != current_node:
                if self.G.edges[(u, v, 0)]['saturation'] > GV.NEXT_CP and self.G.edges[(u, v, 0)]['highway'] not in GV.links:
                    next_edge_congested = True

        if saturation < cp1:
            penalty_rate = beta_1 * saturation
        elif saturation >= cp1 and saturation < cp2:
            penalty_rate = cp1 * beta_1 + (saturation - cp1) * beta_2
        elif saturation >= cp2 and saturation <= 1:
            penalty_rate = cp1 * beta_1 + (cp2 - cp1) * beta_2 + (saturation - cp2) * beta_3
        else:
            penalty_rate = cp1 * beta_1 + (cp2 - cp1) * beta_2 + (1 - cp2) * beta_3

        # Get congestion delay
        congest_delay = penalty_rate * base_delay

        # Get total delay and add it to the dictionary
        delay_sum = base_delay + signal_delay + congest_delay
        self.G.edges[edge]['total_delay'] = delay_sum
        
        if self.G.edges[edge]['alive'] == True:
            # Get expected delay that approximate D_t
            if next_edge_congested:
                if saturation < cp1:
                    exp_penalty_rate = beta_1 * saturation
                elif saturation >= cp1 and saturation <= 1:
                    exp_penalty_rate = cp1 * beta_1 + (saturation - cp1) * beta_3
                else:
                    exp_penalty_rate = cp1 * beta_1 + (1 - cp1) * beta_3                    
                exp_congest_delay = exp_penalty_rate * base_delay
                exp_delay_sum = base_delay + signal_delay + exp_congest_delay

                self.G.edges[edge]['expected_delay'] = exp_delay_sum
            else:
                self.G.edges[edge]['expected_delay'] = delay_sum
        else:
            self.G.edges[edge]['expected_delay'] = float('inf')
            
# Vehicle instance
class Vehicle:
    def __init__(self, identifier, gen_time, src, dst, path):
        # Basic stats
        self.identifier = identifier
        self.gen_time = gen_time
        self.src = src
        self.dst = dst
        
        # dynamic stats
        self.path = path
        self.e_idx = 0
        self.entry_time = gen_time
        self.wait_time = 0 
        self.edge_delay = None
        self.edge_sat = None
        self.arrival_time = None
        self.trapped = False

# Moving process
class Moving_Process(object): 
    def __init__(self, env, G, traffic_generator):
        self.env = env
        self.interval = GV.MOVE_INTV
        self.finished = []
        self.G = G   
        self.tg = traffic_generator
        
        self.v_num = [0]

        # Refine the number of lanes of each edge
        double_lane = ['East Covell Boulevard', 'West Covell Boulevard',
                      'Covell Boulevard', 'Russell Boulevard',
                      'East Covell Boulevard;West Covell Boulevard']
        triple_lane = []
        lane_dic = {}

        for edge in self.G.edges:
            # Get the number of lane
            edge_data = self.G.get_edge_data(edge[0], edge[1], edge[2])
            num_lane = edge_data.get('lanes')

            if isinstance(num_lane, list):
                num_lane = int(max(num_lane))
            elif isinstance(num_lane, str):
                num_lane = int(num_lane)
            elif num_lane == None: 
                num_lane = 1


            # Find the edges that have designated number of lanes
            road_name = edge_data.get('name')
            if isinstance(road_name, list):
                road_name = road_name[0]

            tem_lane_num = 0
            if road_name in double_lane :
                tem_lane_num = 2
            elif road_name in triple_lane :
                tem_lane_num = 3
            else : tem_lane_num = 1

            final_num = max(num_lane, tem_lane_num)
            lane_dic[edge] = final_num

        nx.set_edge_attributes(self.G, lane_dic, 'lanes')

        self.action = env.process(self.run())

    def run(self):
        while True:
            yield self.env.timeout(self.interval)

            # Update 'total_delay' attribute of all the edges
            for edge in self.G.edges:
                self.update_delay(edge)
            
            # Move vehicles

            # Get randomly ordered edges
            edges = list(self.G.edges)
            random_order = random.sample(edges, len(edges))

            for edge in random_order:
              q = self.tg.Q_dic[edge]
              stuck = False

              if len(q) > 0:
                for vehicle in q:
                    vehicle.wait_time += self.interval
                    current_edge = (vehicle.path[vehicle.e_idx], vehicle.path[vehicle.e_idx + 1], 0)

                    last_edge = False
                    try :
                        next_edge = (vehicle.path[vehicle.e_idx + 1], vehicle.path[vehicle.e_idx + 2], 0)
                        next_sat = self.G.edges[next_edge]['saturation']
                    except Exception:
                        last_edge = True
                    
                    if not stuck: # Is this vehicle at the top of the queue?
                        if vehicle.wait_time >= vehicle.edge_delay: # Did the vehicle on top of the queue finish travel the edge?
                            # Check if current edge is the last edge
                            if last_edge:
                                vehicle.arrival_time = self.env.now
                                self.finished.append(vehicle)
                                self.vehicle_exit(current_edge, None, vehicle, last_edge)
                            
                            # Move to next edge if it is not full
                            elif next_sat < 1:                                    
                                # finish traveling current edge
                                self.vehicle_exit(current_edge, next_edge, vehicle)
                                vehicle.e_idx += 1 
                                    
                                # put the vehicle into the next edge    
                                self.vehicle_entry(next_edge, vehicle)
                                self.G.edges[next_edge]['edge_cnt'] += 1
                                vehicle.wait_time = 0
                            else:
                                stuck = True
                        else:
                            stuck = True
                            
            epsilon = 0.01
            
            log_time = self.env.now % 20
            print_time = self.env.now % 100
            
            if log_time < epsilon or abs(log_time) > 20 - epsilon: 
                # Log the number of vehicles in the network in every 20 seconds
                vn = 0
                for queue in self.tg.Q_dic.values():
                    vn += len(queue)
                
                self.v_num.append(vn)
                
            if print_time < epsilon or abs(print_time) > 100 - epsilon: print("Simulation time: {0}".format(self.env.now))
            
    def vehicle_entry(self, edge, vehicle):
        # Add vehicle to the queue of selected edge
        q = self.tg.Q_dic[edge]
        q.append(vehicle)
        vehicle.entry_time = self.env.now
        vehicle.edge_delay = self.G.edges[edge]['total_delay']
        vehicle.edge_sat = self.G.edges[edge]['saturation']
        
        # Update peak traffic and saturation rate of the edge
        trf_len = len(q)
        if trf_len > self.G.edges[edge]['peak_traffic']:
            self.G.edges[edge]['peak_traffic'] = trf_len
        self.G.edges[edge]['saturation'] = (trf_len * GV.VEHICLE_LENGTH) / (self.G.edges[edge]['length'] * self.G.edges[edge]['lanes'])
        
        # Update edge delay
        self.update_delay(edge)

    def vehicle_exit(self, edge, next_edge, vehicle, last_edge=False):
        # Remove vehicle to the queue of selected edge
        q = self.tg.Q_dic[edge]
        q.remove(vehicle)

        # Update delay log of the exiting edge
        delay = self.env.now - vehicle.entry_time
        entry_sat = vehicle.edge_sat
        if next_edge is not None:
            next_sat = self.G.edges[next_edge]['saturation']
        else:
            next_sat = None
        log = (entry_sat, next_sat, delay)
        self.tg.delay_dic[edge].append(log)

        # Update saturation rate of the edge
        trf_len = len(q)
        self.G.edges[edge]['saturation'] = (trf_len * GV.VEHICLE_LENGTH) / (self.G.edges[edge]['length'] * self.G.edges[edge]['lanes'])
        
        # Update edge delay
        self.update_delay(edge)
    
    def update_delay(self, edge):
        base_delay = self.G.edges[edge]['travel_time']
        signal_delay = 0
        congest_delay = 0

        edge_type = self.G.edges[edge]['highway']
        saturation = self.G.edges[edge]['saturation']
        edge_len = self.G.edges[edge]['length']
        beta_1 = GV.BETA_1
        beta_2 = GV.BETA_2
        beta_3 = GV.BETA_3
        cp1 = GV.CP_1
        cp2 = GV.CP_2

        if edge_type == 'primary':
            signal_delay = 10

        elif edge_type == 'secondary':
            signal_delay = 10

        elif edge_type == 'tertiary':
            signal_delay = 6
                
        elif edge_type == 'residential':
            signal_delay = 4

        # Get penalty rate    

        # Is any of possible next edges congested?
        next_edge_congested = False
        (current_node, next_node, key) = edge

        for (u, v) in self.G.out_edges(next_node):
            if v != current_node:
                if self.G.edges[(u, v, 0)]['saturation'] > GV.NEXT_CP and self.G.edges[(u, v, 0)]['highway'] not in GV.links:
                    next_edge_congested = True

        if saturation < cp1:
            penalty_rate = beta_1 * saturation
        elif saturation >= cp1 and saturation < cp2:
            penalty_rate = cp1 * beta_1 + (saturation - cp1) * beta_2
        elif saturation >= cp2 and saturation <= 1:
            penalty_rate = cp1 * beta_1 + (cp2 - cp1) * beta_2 + (saturation - cp2) * beta_3
        else:
            penalty_rate = cp1 * beta_1 + (cp2 - cp1) * beta_2 + (1 - cp2) * beta_3

        # Get congestion delay
        congest_delay = penalty_rate * base_delay

        # Get total delay and add it to the dictionary
        delay_sum = base_delay + signal_delay + congest_delay
        self.G.edges[edge]['total_delay'] = delay_sum
        
        if self.G.edges[edge]['alive'] == True:
            # Get expected delay that approximate D_t
            if next_edge_congested:
                if saturation < cp1:
                    exp_penalty_rate = beta_1 * saturation
                elif saturation >= cp1 and saturation <= 1:
                    exp_penalty_rate = cp1 * beta_1 + (saturation - cp1) * beta_3
                else:
                    exp_penalty_rate = cp1 * beta_1 + (1 - cp1) * beta_3                    
                exp_congest_delay = exp_penalty_rate * base_delay
                exp_delay_sum = base_delay + signal_delay + exp_congest_delay

                self.G.edges[edge]['expected_delay'] = exp_delay_sum
            else:
                self.G.edges[edge]['expected_delay'] = delay_sum
        else:
            self.G.edges[edge]['expected_delay'] = float('inf')

# Rerouting process
class Reroute_Process(object): 
    def __init__(self, env, G, traffic_generator):
        self.env = env
        self.interval = GV.REROUTE_INTV
        self.G = G   
        self.tg = traffic_generator
        self.action = env.process(self.run())

    def run(self):
        while True:
            yield self.env.timeout(self.interval)

            for edge in self.G.edges:
                q = self.tg.Q_dic[edge]
                for vehicle in q:
                    # the index of the node that this vehicle is moving toward on this edge
                    next_node_idx = vehicle.e_idx + 1

                    next_node = vehicle.path[next_node_idx]
                    left_path = vehicle.path[next_node_idx:]
                    new_path = nx.shortest_path(self.G, next_node, vehicle.dst, weight='expected_delay')

                    if left_path != new_path: # reroute if there is a shorter path
                        history = vehicle.path[:next_node_idx]
                        new_route = history + new_path
                        vehicle.path = new_route

# Visualization process
class Vis_Process(object): 
    def __init__(self, env, G):
        self.env = env
        self.interval = GV.VIS_INTV
        self.G = G   
        self.action = env.process(self.run())
        
        # Add a legend
        self.sat_cmap = plt.cm.get_cmap('viridis')
        self.sat_norm = plt.Normalize(vmin=0, vmax=1, clip=True)
        self.sat_sm = cm.ScalarMappable(norm=self.sat_norm, cmap=self.sat_cmap)
        self.sat_sm.set_array([])
        
    def run(self):
        while True:
            yield self.env.timeout(self.interval)
            
            plt.ioff()

            # Visualize saturation

            # Get edge colors with custom function to clip saturation value over 1
            sat_vals = pd.Series(nx.get_edge_attributes(self.G, 'saturation'))
            sat_ec = sat_vals.map(self.sat_sm.to_rgba)

            # Plot a map
            sat_fig, ax = ox.plot_graph(self.G, figsize=(40,24), node_size=0, edge_linewidth=3,
                                    edge_color=sat_ec, show=False, close=False, bgcolor='black')

            # Color legend            
            cb = sat_fig.colorbar(self.sat_sm, fraction=0.05, pad=0.04, orientation='vertical')
            cb.set_label('Saturation rate', fontsize = 50, fontname='Times New Roman', rotation=270, labelpad=70)
            cb.ax.tick_params(labelsize=25)
            
            # sim. time legend
            label = "Simulation time: " + str(self.env.now)
            extra = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
            plt.legend([extra], [label], prop={'size': 25})
            
            # Save figure
            sat_fig.savefig(GV.IMG_DIR + 'sat_' + str(self.env.now) + '.jpg', format='jpg')
            plt.close('all')
    
# Disruption on edges
class Edge_Attack(object): 
    def __init__(self, env, G, traffic_generator):
        self.env = env
        self.atk_rate = GV.ATK_RATE
        self.G = G
        self.hist = {}
        self.tg = traffic_generator
        self.atk_cnt = 0

        # Term edge count after idx-th attack
        self.term_edge_cnt = []
        
        # Colormap for delay chart
        self.cmap = plt.cm.get_cmap('coolwarm')
        self.norm = plt.Normalize(vmin=0, vmax=1, clip=True)
        self.sm = cm.ScalarMappable(norm=self.norm, cmap=self.cmap)
        self.sm.set_array([])

        self.action = env.process(self.run())
        
    def run(self):       
        yield self.env.timeout(GV.WARMING_UP) # defer attack to the end of warming-up period
        nx.set_edge_attributes(self.G, 0, 'edge_cnt')
        
        while self.atk_cnt < 5: # End process after 5 disruptions
            yield self.env.timeout(self.atk_rate)
                       
            # Find the most important (=frequently visited) edge
            # Get visit counts of all the edges
            cnt = nx.get_edge_attributes(self.G, 'edge_cnt')
            
            # Delete previously removed edges and get the most frequently visited edge
            removed_edges = list(self.hist.values())
            for e in removed_edges:
                del cnt[e]
            vul_edge = max(cnt, key=cnt.get)
            
            # Change expected time cost to infinity
            self.G.edges[vul_edge]['expected_delay'] = float('inf')
            self.G.edges[vul_edge]['alive'] = False
            
            # Log attack history (edge, atk_time)
            self.hist[self.env.now] = vul_edge

            # Reset edge count to get count per attack rate in next iteration
            term_cnt = {}
            for edge in self.G.edges:
                term_cnt[edge] = self.G.edges[edge]['edge_cnt']
                self.G.edges[edge]['acc_edge_cnt'] += self.G.edges[edge]['edge_cnt']
            nx.set_edge_attributes(self.G, 0, 'edge_cnt')
            self.term_edge_cnt.append(term_cnt)

            # Visualize edge delay statistics
            self.vis_edge_delay(vul_edge)
            
            # Reroute to avoid the disrupted edge, as later as possible before arriving the edge
            self.reroute()

    def reroute(self): 
        for edge in self.G.edges:
            q = self.tg.Q_dic[edge]
            for vehicle in q:
                # the index of the node that this vehicle is moving toward on the current edge
                next_node_idx = vehicle.e_idx + 1

                next_node = vehicle.path[next_node_idx]
                left_path = vehicle.path[next_node_idx:]
                new_path = nx.shortest_path(self.G, next_node, vehicle.dst, weight='expected_delay')

                if left_path != new_path: # reroute if there is a shorter path
                    history = vehicle.path[:next_node_idx]
                    new_route = history + new_path
                    vehicle.path = new_route
                        
    def vis_edge_delay(self, vul_edge):
        # Draw delay computed by heuristic for comparison        
        base_delay = self.G.edges[vul_edge]['travel_time']
        signal_delay = 0
        congest_delay = 0
        edge_len = self.G.edges[vul_edge]['length']

        edge_type = self.G.edges[vul_edge]['highway']
        if edge_type == 'primary':
            signal_delay = 10

        elif edge_type == 'secondary':
            signal_delay = 10

        elif edge_type == 'tertiary':
            signal_delay = 6

        total_delay = []
        total_delay2 = []
        x = np.linspace(0,1,101)

        beta1 = GV.BETA_1
        beta2 = GV.BETA_2
        beta3 = GV.BETA_3
        cp1 = GV.CP_1
        cp2 = GV.CP_2

        for i in x:
            pr = 0
            pr2 = 0

            if i < cp1: pr = i*beta1
            elif cp1 <= i and i < cp2: pr = cp1*beta1 + (i-cp1)*beta2
            elif i >= cp2: pr = cp1*beta1 + (cp2-cp1)*beta2 + (i-cp2)*beta3

            if i < cp1: pr2 = i*beta1
            elif cp1 <= i : pr2 = cp1*beta1 + (i-cp1)*beta3

            congest_delay = pr * base_delay
            congest_delay2 = pr2 * base_delay

            d = base_delay + signal_delay + congest_delay
            d2 = base_delay + signal_delay + congest_delay2

            total_delay.append(d)
            total_delay2.append(d2)

        self.atk_cnt += 1
        title = 'Delay on the ' + str(self.atk_cnt) + '-th disrupted edge'

        plt.plot(x, total_delay2, c='#ff7f0e')
        plt.plot(x, total_delay, c='black')

        # Scatter log item
        delay_logs = self.tg.delay_dic[vul_edge]

        for (sat, next_edge_sat, delay) in delay_logs:
            if sat > 1:
                sat = 1

            if next_edge_sat is not None:    
                rgba_col = self.sm.to_rgba(next_edge_sat)
                hex_col = colors.to_hex(rgba_col)
                plt.scatter(sat, delay, c=hex_col, alpha=0.8)
            else:
                plt.scatter(sat, delay, c='black', alpha=0.8)

        plt.colorbar(self.sm, label='Next edge saturation rate')            
        plt.xlabel('Saturation rate') 
        plt.ylabel('Delay')

        plt.savefig(GV.IMG_DIR + 'Delay on the ' + str(self.atk_cnt) +'-th disrupted edge.png')
        plt.close('all')

In [34]:
### test weighted OD pair selection ###

# load road network from a binary pickle file 
fname = 'Davis_simplified_graph.pkl'
rf = open(fname,"rb")
G = pickle.load(rf)
rf.close()

nodes= list(G.nodes)
weights= [G.nodes[node]['aggr_node_num'] for node in nodes]

nx.set_node_attributes(G, 0, 'sample_cnt')

weights_sum = sum(weights) *3
multiple_weights = [weight*3 for weight in weights]

cnt =0
while True:
    picked_node = random.choices(nodes, weights=weights)[0]
    
    G.nodes[picked_node]['sample_cnt'] += 1
    if cnt > weights_sum: break
    else:
        cnt += 1

In [35]:
random.seed(GV.SEED)

# load road network from a binary pickle file 
fname = 'Davis_simplified_graph.pkl'
rf = open(fname,"rb")
G = pickle.load(rf)
rf.close()

# Change edge key to 1 (This simplified graph does not have multiple edges)
nonZero_keys = []
for edge in G.edges(keys=True, data=True):
    u,v,k,d = edge
    if k != 0:
        nonZero_keys.append(edge)
for u,v,k,d in nonZero_keys:
    G.add_edge(u, v, key=0, **d)
    G.remove_edge(u, v, key=k)

# Refine edge type
etype_val = {'motorway':0, 'motorway_link':1, 'primary':2, 'primary_link':3,
               'secondary':4, 'secondary_link':5, 'tertiary':6, 'tertiary_link':7,
               'residential':8, 'unclassified':9}

for edge in G.edges:
    edge_type = G.edges[edge]['highway']
    
    # Set the most slow edge type for consolidated edges
    if type(edge_type) == list:
        edge_type = sorted(edge_type, reverse=True, key = lambda d: etype_val[d])
        edge_type = edge_type[0]
        G.edges[edge]['highway'] = edge_type

# Set default speed of vehicles on each type of road
hwy_speeds = {'motorway': 110,
            'motorway_link': 60,
            'primary': 60,
            'primary_link': 60,
            'secondary': 50,
            'secondary_link': 50,
            'tertiary': 40,
            'tertiary_link': 40,
            'residential': 40,
            'unclassified': 40}

# Plug in to the graph. 'speed_kph' attribute is added to each edge and set to the default speed.
G = ox.add_edge_speeds(G, hwy_speeds)

# Add 'travel_time' property to each node, which is 'length' divided by 'speed_kph'
G = ox.add_edge_travel_times(G)

# Add signal light delay on the travel_time attributes
for edge in G.edges:
    edge_type = G.edges[edge]['highway']
    edge_len = G.edges[edge]['length']
    signal_delay = 0

    if edge_type == 'primary':
        signal_delay = 10

    elif edge_type == 'secondary':
        signal_delay = 10

    elif edge_type == 'tertiary':
        signal_delay = 6

    elif edge_type == 'residential':
        signal_delay = 4
            
    G.edges[edge]['travel_time'] += signal_delay

travel_time = nx.get_edge_attributes(G, 'travel_time')
nx.set_edge_attributes(G, travel_time,'total_delay') # initialize 'total_delay' attributes
nx.set_edge_attributes(G, travel_time,'expected_delay') # initialize 'expected_delay' attributes

# Initialize other edge attributes
nx.set_edge_attributes(G, 0, 'peak_traffic')
nx.set_edge_attributes(G, 0, 'saturation')
nx.set_edge_attributes(G, 0, 'edge_cnt')
nx.set_edge_attributes(G, 0, 'acc_edge_cnt')
nx.set_edge_attributes(G, True, 'alive')

# Set environment
env = simpy.Environment()

tg = Traffic_Gen(env, G) 
mv = Moving_Process(env, G, tg)

if GV.ATK_RATE is not None:
    edge_atk = Edge_Attack(env, G, tg) 

if GV.VIS_INTV is not None:
    vis = Vis_Process(env, G)

if GV.REROUTE_INTV is not None:
    rr = Reroute_Process(env, G, tg)
        
start_time = time.time()
env.run(until=GV.SIM_TIME)
end_time = time.time()
elapsed_time = end_time - start_time
print('Time cost for the simulation:', elapsed_time, 'sec')

Simulation time: 100.00000000000088
Time cost for the simulation: 27.231456518173218 sec


In [36]:
# Plot edge counts on the map
edge_cnt = []
for e in G.edges:
    edge_cnt.append(G.edges[e]['acc_edge_cnt'])

cmap_name = 'coolwarm'
#col_bin = 10
min_col = 0.3
max_col = 1
    
edge_cnt_ec = ox.plot.get_edge_colors_by_attr(G, "acc_edge_cnt", cmap=cmap_name, start=min_col, stop=max_col, na_color='none')

edge_cnt_fig, edge_cnt_ax = ox.plot_graph(G, figsize=(40,24), node_color='b', node_size=0,
                                edge_linewidth=3, edge_color=edge_cnt_ec, show=False, close=False, bgcolor='white')

# Add a legend
cmap = plt.cm.get_cmap(cmap_name)
cmap = colors.LinearSegmentedColormap.from_list(
    'trunc({name},{a:.2f},{b:.2f})'.format(name=cmap.name, a=min_col, b=max_col),
    cmap(np.linspace(min_col, max_col, cmap.N)))
norm = plt.Normalize(vmin=int(min(edge_cnt)), vmax=int(max(edge_cnt)))
sm = cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])

cb = edge_cnt_fig.colorbar(sm, fraction=0.05, pad=0.04, orientation='vertical')
cb.set_label('Edge visit count', fontsize = 50, fontname='Times New Roman', rotation=270, labelpad=70)
cb.ax.tick_params(labelsize=25)

plt.savefig(GV.IMG_DIR + 'Edge_count_map.jpg', format='jpg')
plt.close('all')

# Print vehicle completion status
print(tg.vehicle_number, 'vehicles are generated')
print(len(mv.finished), 'vehicles completed travel')

# Show added delays
total_delay = nx.get_edge_attributes(G, 'total_delay')
travel_time = nx.get_edge_attributes(G, 'travel_time')

values = []
for edge in G.edges:
    values.append(total_delay[edge] - travel_time[edge])

sorted_values = sorted(values, reverse=True)   

plt.title('Additional delay per edges')
plt.xlabel('Rank') 
plt.ylabel('Delay')
plt.plot(sorted_values)
plt.savefig(GV.IMG_DIR + 'Added_delay_chart.jpg', format='jpg')
plt.close('all')

149 vehicles are generated
0 vehicles completed travel


In [37]:
# Plot attacked edges on the map

if GV.ATK_RATE is not None:
    hist_list = list(edge_atk.hist.values())

    removed = []
    for e in hist_list:
        removed.append( [e[0], e[1]] )

    c_pool = ['royalblue','red','violet','orange','aquamarine']
    rc = []

    for i in range(0, len(removed)):
        rc.append(c_pool[i])

    atk_fig, atk_ax = ox.plot.plot_graph_routes(G, removed, figsize=(30,18), route_colors=rc, 
                                                route_linewidth=6, route_alpha=0.8, orig_dest_size=100,
                                                ax=None, node_size=20, node_color='black',
                                                edge_linewidth=0.5, edge_color='black', 
                                                show=False, close=False, bgcolor='white')

    for i, e in enumerate(removed):
        start_node = e[0]
        end_node = e[1]
        start_coord = (G.nodes[start_node]['x'] - 0.0006, G.nodes[start_node]['y'] - 0.0006)
        end_coord =  (G.nodes[end_node]['x'] - 0.0006, G.nodes[end_node]['y'] - 0.0006)

        # Annotate node ID if needed
        #atk_ax.annotate(start_node, xy=start_coord, color='white', weight='bold')
        #atk_ax.annotate(end_node, xy=end_coord, color='blue', weight='bold')

        middle_coord = (start_coord[0] + 0.5*(end_coord[0] - start_coord[0]), start_coord[1] + 0.5*(end_coord[1] - start_coord[1]))

        arrow_len = 0.001

        x_s = start_coord[0]
        y_s = start_coord[1]
        x_e = end_coord[0]
        y_e = end_coord[1]
        x_m = middle_coord[0]
        y_m = middle_coord[1]

        x_diff = x_s - x_m
        y_diff = y_s - y_m
        grad = y_diff / x_diff

        x_sign = x_diff / abs(x_diff)
        y_sign = y_diff / abs(y_diff)

        arrow_start_coord = (middle_coord[0] + 0.3*(start_coord[0] - middle_coord[0]), middle_coord[1] + 0.3*(start_coord[1] - middle_coord[1]))
        arrow_end_coord = (middle_coord[0] + 0.3*(end_coord[0] - middle_coord[0]), middle_coord[1] + 0.3*(end_coord[1] - middle_coord[1]))    
        dist = (((arrow_start_coord[0]-arrow_end_coord[0])**2 + (arrow_start_coord[1] - arrow_end_coord[1])**2)**0.5)

        if dist < 0.003:
                x_mov = x_sign*((arrow_len**2 / (1 + grad**2))**0.5)
                y_mov = y_sign*((arrow_len**2 - x_mov**2) ** 0.5)
                arrow_start_coord = (middle_coord[0] + x_mov, middle_coord[1] + y_mov)
                arrow_end_coord = (middle_coord[0] - x_mov, middle_coord[1] - y_mov)

        atk_ax.annotate('', xy= arrow_end_coord, color='blue', weight='bold', xytext= arrow_start_coord, textcoords='data', arrowprops=dict(color='yellow', headwidth=10, headlength=10, lw=1))

    # Add legend
    line_label, = plt.plot([0, 0, 0], label='1st', c='none', linewidth = 0)
    line_1st, = plt.plot([0, 0, 0], label='1st', c='royalblue', linewidth = 6)
    line_2nd, = plt.plot([0, 0, 0], label='2nd', c='red', linewidth = 6)
    line_3rd, = plt.plot([0, 0, 0], label='3rd', c='violet', linewidth = 6)
    line_4th, = plt.plot([0, 0, 0], label='4th', c='orange', linewidth = 6)
    line_5th, = plt.plot([0, 0, 0], label='5th', c='aquamarine', linewidth = 6)


    plt.legend([line_label, line_1st, line_2nd, line_3rd, line_4th, line_5th], ['Top-5 edges', '1st', '2nd', '3rd', '4th', '5th'], prop={'size': 20})

    plt.savefig(GV.IMG_DIR + 'Attacked_edges_map.png')
    plt.close('all')
    
    disrupted = edge_atk.hist.values()

    disrupted_fname = GV.IMG_DIR + 'disrupted.txt'
    with open(disrupted_fname, "w") as output:
        for value in disrupted:
            output.write(str(value) + '\n')

In [38]:
# Save travel time
finished_travelTime = []
for vehicle in mv.finished:
    travelTime = vehicle.arrival_time - vehicle.gen_time
    finished_travelTime.append(travelTime)

sorted_finished_travelTime = sorted(finished_travelTime, reverse=True)
fd_fname = GV.IMG_DIR + 'finished_travelTime.txt'
with open(fd_fname, "w") as output:
    for value in sorted_finished_travelTime:
        output.write(str(value) + '\n')

onTheWay_travelTime = []
for edge in G.edges:
    for vehicle in tg.Q_dic[edge]:
        travelTime = GV.SIM_TIME - vehicle.gen_time
        onTheWay_travelTime.append(travelTime)
        
sorted_onTheWay_travelTime = sorted(onTheWay_travelTime, reverse=True)
otw_fname = GV.IMG_DIR + 'onTheWay_travelTime.txt'
with open(otw_fname, "w") as output:
    for value in sorted_onTheWay_travelTime:
        output.write(str(value) + '\n')
        
# Save general stats
stat_fname = GV.IMG_DIR + 'stats.txt'

finished_sum = sum(finished_travelTime)
otw_sum = sum(onTheWay_travelTime)
total_travelTime = finished_sum + otw_sum

with open(stat_fname, "w") as output:
    output.write(str(tg.vehicle_number) + ' vehicles are generated' + '\n')
    output.write(str(len(mv.finished)) + ' vehicles completed travel' + '\n')
    output.write('Travel time sum. of finished travels: ' + str(finished_sum) + '\n')
    output.write('Travel time sum. of vehicles on their way: ' + str(otw_sum) + '\n')
    output.write('Total travel time sum.: ' + str(total_travelTime) + '\n')

In [39]:
# Dump data for debugging
dat_dir = GV.IMG_DIR + "var_dat/"
os.mkdir(dat_dir)

nx.write_gpickle(G, dat_dir + "graph.gpickle")

with open(dat_dir + "Q_dic.p", 'wb') as f:
    pickle.dump(tg.Q_dic, f)
    
with open(dat_dir + "delay_dic.p", 'wb') as f:
    pickle.dump(tg.delay_dic, f)
    
with open(dat_dir + "finished.p", 'wb') as f:
    pickle.dump(mv.finished, f)

with open(dat_dir + "v_num.p", 'wb') as f:
    pickle.dump(mv.v_num, f)

In [40]:
vnum_fname = GV.IMG_DIR + 'vnum_gr-' + str(GV.GEN_RATE) + '_rr-' + str(GV.REROUTE_INTV) +'.txt'
with open(vnum_fname, "w") as output:
    for value in mv.v_num:
        output.write(str(value) + '\n')

In [41]:
num_dp = ((GV.SIM_TIME - 1) / 20) + 1
x = [ i*20 for i in range(int(num_dp)) ]

x2 = [ i*20 for i in range(int(num_dp))]
y = [ t*GV.GEN_RATE for t in x2 ]

#dsrp_x = [800, 1100, 1400, 1700, 2000]

plt.plot(x, mv.v_num, label='Moving vehicles')
plt.plot(x2, y, label='Generated')
#plt.vlines(dsrp_x, 0, 40000, colors='black', linestyles='dotted', label='Disruptions', alpha=0.5)
#plt.vlines(2700, 0, 40000, colors='black', linestyles='dashed', label='Generation ended', alpha=0.9)
        
plt.title('The number of vehicles')
plt.xlabel('Seconds') 
plt.ylabel('Counts')

plt.ylim(top=max(mv.v_num)*1.1, bottom=0)
plt.xlim(left=0)
plt.legend()

plt.savefig(GV.IMG_DIR + 'num_vehicle.png')
plt.close('all')